http://blog.datadive.net/selecting-good-features-part-ii-linear-models-and-regularization/

Many machine learning models have either some inherent internal ranking of features or it is easy to generate the ranking from the structure of the model. This applies to regression models, SVM’s, decision trees, random forests, etc.

## multicollinearity 

https://en.wikipedia.org/wiki/Multicollinearity

In statistics, multicollinearity (also collinearity) is a phenomenon in which two or more predictor variables in a multiple regression model are highly correlated, meaning that one can be linearly predicted from the others with a substantial degree of accuracy. In this situation the coefficient estimates of the multiple regression may change erratically in response to small changes in the model or the data. **Multicollinearity does not reduce the predictive power or reliability of the model as a whole**, at least within the sample data set; it only affects calculations regarding individual predictors. That is, a multiple regression model with correlated predictors can indicate how well the entire bundle of predictors predicts the outcome variable, **but it may not give valid results about any individual predictor, or about which predictors are redundant with respect to others**.

# Regularization

Regularization is a method for adding additional constraints or penalty to a model, with the goal of preventing overfitting and improving generalization. Instead of minimizing a loss function E(X,Y), the loss function to minimize becomes E(X,Y)+α‖w‖, where w is the vector of model coefficients, ‖⋅‖ is typically L1 or L2 norm and α is a tunable free parameter, specifying the amount of regularization

## L1 regularization -- Lasso

 * α∑|wi|
 * L1 regularization produces sparse solutions, inherently performing feature selection.

In [16]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston
  
boston = load_boston()
scaler = StandardScaler()
X = scaler.fit_transform(boston["data"])
Y = boston["target"]
names = boston["feature_names"]
  
lasso = Lasso(alpha=.3)
lasso.fit(X, Y)

print "Lasso model: "
print sorted(zip([round(i,2) for i in lasso.coef_], names))

Lasso model: 
[(-3.71, 'LSTAT'), (-1.76, 'PTRATIO'), (-1.08, 'DIS'), (-0.7, 'NOX'), (-0.24, 'CRIM'), (-0.0, 'AGE'), (-0.0, 'INDUS'), (0.0, 'RAD'), (-0.0, 'TAX'), (0.08, 'ZN'), (0.54, 'CHAS'), (0.63, 'B'), (2.99, 'RM')]


How to choose α（see last section)

Note however that L1 regularized regression is unstable (reason see example below) in a similar way as unregularized linear models are, meaning that the coefficients (and thus feature ranks) can vary significantly even on small data changes when there are correlated features in the data. Which brings us to L2 regularization.

## L2 regularization -- Ridge regression

* α∑wi^2
* it has a different effect from L1-norm, namely it forces the coefficient values to be spread out more equally. For correlated features, it means that they tend to get similar coefficients. Coming back to the example of Y=X1+X2, with strongly correlated X1 and X2, then for L1, the penalty is the same whether the learned model is Y=1∗X1+1∗X2 or Y=2∗X1+0∗X2. In both cases the penalty is 2∗α. For L2 however, the first model’s penalty is 12+12=2α, while for the second model is penalized with 22+02=4α. **The effect of this is that models are much more stable**
* So while L2 regularization **does not perform feature selection the same way as L1** does, it is more useful for ** feature *interpretation* **: a predictive feature will get a non-zero coefficient, which is often not the case with L1

In [24]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import numpy as np
size = 100
 
#We run the method 10 times with different random seeds
for i in range(10):
    print "Random seed %s" % i
    np.random.seed(seed=i)
    X_seed = np.random.normal(0, 1, size)
    X1 = X_seed + np.random.normal(0, .1, size)
    X2 = X_seed + np.random.normal(0, .1, size)
    X3 = X_seed + np.random.normal(0, .1, size)
    Y = X1 + X2 + X3 + np.random.normal(0, 1, size)
    X = np.array([X1, X2, X3]).T
 
 
    lr = LinearRegression()
    lr.fit(X,Y)
    print "Linear model:", 
    print sorted([round(i,2) for i in lr.coef_])
    #print sorted(zip([round(i,2) for i in lasso.coef_], names))
 
    ridge = Ridge(alpha=10)
    ridge.fit(X,Y)
    print "Ridge model:",
    print sorted([round(i,2) for i in ridge.coef_])
    #print sorted(zip([round(i,2) for i in ridge.coef_], names))

Random seed 0
Linear model: [-0.08, 0.73, 2.31]
Ridge model: [0.88, 0.94, 1.06]
Random seed 1
Linear model: [-0.6, 1.15, 2.37]
Ridge model: [0.76, 0.98, 1.07]
Random seed 2
Linear model: [0.32, 0.7, 2.09]
Ridge model: [0.94, 0.97, 1.09]
Random seed 3
Linear model: [0.29, 1.25, 1.49]
Ridge model: [0.92, 1.0, 1.03]
Random seed 4
Linear model: [0.19, 0.77, 2.19]
Ridge model: [0.96, 0.98, 1.1]
Random seed 5
Linear model: [-1.29, 1.59, 2.75]
Ridge model: [0.76, 1.01, 1.14]
Random seed 6
Linear model: [-0.03, 1.2, 1.91]
Ridge model: [0.89, 1.02, 1.09]
Random seed 7
Linear model: [-0.15, 1.47, 1.76]
Ridge model: [0.9, 1.02, 1.04]
Random seed 8
Linear model: [0.08, 1.11, 1.88]
Ridge model: [0.91, 1.01, 1.07]
Random seed 9
Linear model: [0.71, 0.78, 1.36]
Ridge model: [0.9, 0.9, 0.98]


## Summary

This session is about feature selection when features are correlated.

Regularized linear models are a powerful set of tool for feature interpretation and selection. Lasso produces sparse solutions and as such is very useful selecting a strong subset of features for improving model performance. Ridge regression on the other hand can be used for data interpretation due to its stability and the fact that useful features tend to have non-zero coefficients. Since the relationship between the response variable and features in often non-linear, basis expansion can be used to convert features into a more suitable space, while keeping the simple linear models fully applicable.

## How to select hyper parameter
* http://scikit-learn.org/stable/modules/grid_search.html
* using **Cross validation**

### Exhaustive Grid Search
GridSearchCV will use the same shuffling for each set of parameters validated by a single call to its fit method

example code : http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html#sphx-glr-auto-examples-model-selection-grid-search-digits-py

In [27]:
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'kernel': 'rbf', 'C': 10, 'gamma': 0.001}

Grid scores on development set:

0.986 (+/-0.016) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001}
0.959 (+/-0.029) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001}
0.988 (+/-0.017) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}
0.982 (+/-0.026) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001}
0.988 (+/-0.017) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.001}
0.982 (+/-0.025) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.0001}
0.988 (+/-0.017) for {'kernel': 'rbf', 'C': 1000, 'gamma': 0.001}
0.982 (+/-0.025) for {'kernel': 'rbf', 'C': 1000, 'gamma': 0.0001}
0.975 (+/-0.014) for {'kernel': 'linear', 'C': 1}
0.975 (+/-0.014) for {'kernel': 'linear', 'C': 10}
0.975 (+/-0.014) for {'kernel': 'linear', 'C': 100}
0.975 (+/-0.014) for {'kernel': 'linear', 'C': 1000}

Detailed classification report:

The model 

example code 2: http://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html#sphx-glr-auto-examples-model-selection-plot-nested-cross-validation-iris-py

I dont understand

In [26]:
from sklearn.datasets import load_iris
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np

print(__doc__)

# Number of random trials
NUM_TRIALS = 30

# Load the dataset
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

# Set up possible values of parameters to optimize over
p_grid = {"C": [1, 10, 100],
          "gamma": [.01, .1]}

# We will use a Support Vector Classifier with "rbf" kernel
svr = SVC(kernel="rbf")

# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "LabelKFold", "LeaveOneOut", "LeaveOneLabelOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svr, param_grid=p_grid, cv=inner_cv)
    clf.fit(X_iris, y_iris)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=X_iris, y=y_iris, cv=outer_cv)
    nested_scores[i] = nested_score.mean()

score_difference = non_nested_scores - nested_scores

print("Average difference of {0:6f} with std. dev. of {1:6f}."
      .format(score_difference.mean(), score_difference.std()))

# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on Iris Dataset",
          x=.5, y=1.1, fontsize="15")

plt.show()

Automatically created module for IPython interactive environment
Average difference of 0.007742 with std. dev. of 0.007688.


### Randomized Parameter Optimization


http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html#sphx-glr-auto-examples-model-selection-randomized-search-py

In [5]:
# print(__doc__)

import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

# get some data
digits = load_digits()
X, y = digits.data, digits.target

# build a classifier
clf = RandomForestClassifier(n_estimators=20)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

RandomizedSearchCV took 6.05 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.923 (std: 0.001)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 4, 'criterion': 'entropy', 'max_features': 2, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.917 (std: 0.006)
Parameters: {'bootstrap': True, 'min_samples_leaf': 4, 'min_samples_split': 7, 'criterion': 'gini', 'max_features': 5, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.913 (std: 0.010)
Parameters: {'bootstrap': False, 'min_samples_leaf': 8, 'min_samples_split': 6, 'criterion': 'entropy', 'max_features': 6, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.913 (std: 0.008)
Parameters: {'bootstrap': True, 'min_samples_leaf': 2, 'min_samples_split': 5, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}

GridSearchCV took 63.60 seconds for 216 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.93